# Dynamic Message-Passing Approximation Method for Adaptive Mutations on Barabási-Albert Networks 

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import polars as pl
import networkx as nx
import altair as alt
from multiprocess import Pool
from scipy.interpolate import interp1d


from origins_of_the_fittest.network_construction.load_example_networks import (
    load_network_barabasi_albert_Meq4N,
)

from origins_of_the_fittest.simulation.spreading.one_layer_dijkstra import (
    arrival_times_dijkstra,
)

from origins_of_the_fittest.approximation.distance_message_passing import (
    si_dynamic_message_passing_unweighted,
)

from origins_of_the_fittest.approximation.tau_h_integrals import (
    compute_H_integral_normalized,
)

from origins_of_the_fittest.approximation.fitness_variance_homogeneous import (
    dotr_steady_state,
)

from origins_of_the_fittest.approximation.pi_computation import H2pi

from origins_of_the_fittest.utility.plot_altair import altair_helvetica_theme

In [2]:
# Configure Altair theme
# switch between "light" and "dark" for better visibility in light/dark mode
altair_helvetica_theme("dark")

## Load Data and Network

- Barabási-Albert network with N = 1024 nodes and M = 4N edges
- Simulation data from 1000 runs with mutation rate μ = 10⁻³, s_eff = 0.2


In [3]:
# Load network data
A_ba1024 = load_network_barabasi_albert_Meq4N(1024)


# Load simulation data
file_path = "df_simulation_tau_pi_ba1024mu1e-3.parquet"
df_adaptive_ba1024 = (
    pd.read_parquet(file_path).sort_values("origin").reset_index(drop=True)
)

df_adaptive_ba1024.head()

,origin,tau_simulation,tau_simulation_inv,pi_simulation
0,0,8.724844,0.114615,0.004312
1,1,11.109534,0.090013,0.002620
2,2,10.512971,0.095121,0.003024
3,3,10.298010,0.097106,0.003115
4,4,11.983190,0.083450,0.002387


In [4]:
alt.Chart(df_adaptive_ba1024).mark_point().encode(
    x=alt.X("tau_simulation_inv", scale=alt.Scale(type="linear", zero=False)).title(
        "1 / τ (simulation)"
    ),
    y=alt.Y("pi_simulation", scale=alt.Scale(type="linear")).title("π (simulation)"),
)

alt.Chart(...)

## Parameters

In [5]:
A_values = A_ba1024.values
N = A_values.shape[0]

lambda_eff = 1.0
mu = 1e-3
s0 = 0.2

## Method 2: Dynamic Message Passing (DMP) Approximation with $s_{eff} \approx s_0$



In [6]:
def _compute_Y_for_origin(j: int):
    """
    Compute H[:, j] for a single origin node j.
    Returns (j, H_col) so we can place results in the right column.
    """
    # Initial condition: only node j is infected
    Y0 = np.zeros(N, dtype=float)
    Y0[j] = 1.0

    # Run DMP to get spreading dynamics (using normalized formulation)
    Y_history, ts = si_dynamic_message_passing_unweighted(
        A_values,
        lambda_eff=1 / 8,
        Y0=Y0,
        epsilon_stop=1e-8,
        step_change=0.02,
    )

    Y_func = interp1d(
        ts,
        Y_history,
        axis=0,
        kind="linear",
        bounds_error=False,
        fill_value=(np.zeros(N), np.ones(N)),
    )

    return j, Y_func


print("Computing Y(t) for all origin nodes using DMP approximation (parallel)...")

Y_func_dmp = [None] * N

with Pool() as pool:
    # imap_unordered yields results as they finish
    for count, (j, Y_func) in enumerate(
        pool.imap_unordered(_compute_Y_for_origin, range(N)),
        start=1,
    ):
        Y_func_dmp[j] = Y_func

        if count % 128 == 0 or count == N:
            print(f"  Progress: {count}/{N}")

Computing Y(t) for all origin nodes using DMP approximation (parallel)...


  Progress: 128/1024
  Progress: 256/1024
  Progress: 384/1024
  Progress: 512/1024
  Progress: 640/1024
  Progress: 768/1024
  Progress: 896/1024
  Progress: 1024/1024


In [7]:
print("Computing H matrix")

H_matrix_dmp = np.zeros((N, N), dtype=float)
for j in range(N):

    # H[:, j] = destinations given origin j
    H_col = compute_H_integral_normalized(
        Y_func_dmp[j],
        mu=mu,
        s_eff=s0,
        t_max_est=10 / N / mu,
    )

    H_matrix_dmp[:, j] = H_col


df_H_dmp = pd.DataFrame(
    H_matrix_dmp,
    index=pd.Index(range(N), name="i"),  # i = destination
    columns=pd.Index(range(N), name="j"),  # j = origin
)

print("Done!")

Computing H matrix
Done!


In [8]:
# Convert H matrix to establishment probabilities
df_pi_dmp = H2pi(df_H_dmp).to_frame(name="pi_dmp")

In [9]:
alt.Chart(df_adaptive_ba1024.join(df_pi_dmp)).mark_point().encode(
    x=alt.X("pi_dmp")
    .title("π (Approximation, DMP)")
    .scale(type="linear", domain=[0, 0.006]),
    y=alt.Y("pi_simulation")
    .title("π (Simulation)")
    .scale(type="linear", domain=[0, 0.006]),
).properties(
    title=alt.TitleParams(
        "DMP Approximation π vs Simulation π",
        subtitle="The DMP approximation of π is a good, fast predictor of simulation π",
        subtitleColor="#666",
    )
)

alt.Chart(...)

In [10]:
correlation = (
    df_adaptive_ba1024.drop(columns=["tau_simulation", "tau_simulation_inv"])
    .join(df_pi_dmp)
    .corr(method="pearson")
    .loc["pi_simulation", "pi_dmp"]
)
print(
    f"Pearson correlation between π (DMP Approximation) and π (Simulation): {correlation}"
)

Pearson correlation between π (DMP Approximation) and π (Simulation): 0.9965989675022424


## Method 2: Dynamic Message Passing (DMP) Approximation with estimated $s_{eff}$

In [11]:
graph_dict_ba1024 = nx.to_dict_of_lists(nx.from_numpy_array(A_values))

arrival_times_2d = np.zeros((N, N), dtype=float)

for j in range(N):
    arrival_times_2d[:, j] = list(
        arrival_times_dijkstra(
            graph=graph_dict_ba1024, source=j, lambda_eff=s0 / 8, rng_seed=j
        ).values()
    )

dotr, s_eff_estimate, _ = dotr_steady_state(
    arrival_times_2d,
    mu=mu,
    lower_bracket=s0 + 0.01,
    upper_bracket=1.0,
    s0=s0,
    use_transformed_arrival=True,
)
print(f"Estimated s_eff: {s_eff_estimate}, vs s0: {s0}")

Estimated s_eff: 0.5127314520873546, vs s0: 0.2


In [12]:
print("Computing H matrix")

H_matrix_dmp_seff = np.zeros((N, N), dtype=float)
for j in range(N):

    # H[:, j] = destinations given origin j
    H_col = compute_H_integral_normalized(
        Y_func_dmp[j],
        mu=mu,
        s_eff=s_eff_estimate,
        t_max_est=10 / N / mu,
    )

    H_matrix_dmp_seff[:, j] = H_col


df_H_dmp_seff = pd.DataFrame(
    H_matrix_dmp_seff,
    index=pd.Index(range(N), name="i"),  # i = destination
    columns=pd.Index(range(N), name="j"),  # j = origin
)

print("Done!")

Computing H matrix
Done!


In [13]:
# Convert H matrix to establishment probabilities
df_pi_dmp_seff = H2pi(df_H_dmp_seff).to_frame(name="pi_dmp_seff")

In [14]:
(
    alt.Chart(
        pl.DataFrame(
            {
                "pi_simulation": [0.0, 0.006],
                "pi_value": [0.0, 0.006],
            }
        )
    )
    .mark_line(stroke="#888", strokeDash=[3, 3], strokeCap="round")
    .encode(
        x=alt.X("pi_value")
        .title("π (Approximation, DMP)")
        .scale(type="linear", domain=[0, 0.006]),
        y=alt.Y("pi_simulation")
        .title("π (Simulation)")
        .scale(type="linear", domain=[0, 0.006]),
    )
    + alt.Chart(
        df_adaptive_ba1024.drop(columns=["tau_simulation", "tau_simulation_inv"])
        .join(df_pi_dmp.rename(columns={"pi_dmp": "π (DMP, s_eff≈s0)"}))
        .join(
            df_pi_dmp_seff.rename(
                columns={f"pi_dmp_seff": f"π (DMP, s_eff≈{s_eff_estimate:.2f})"}
            )
        )
        .set_index(["origin", "pi_simulation"])
        .stack()
        .rename("pi_value")
        .reset_index()
        .rename(columns={"level_2": "method"})
    )
    .mark_point()
    .encode(
        x=alt.X("pi_value")
        .title("π (Approximation, DMP)")
        .scale(type="linear", domain=[0, 0.006]),
        y=alt.Y("pi_simulation")
        .title("π (Simulation)")
        .scale(type="linear", domain=[0, 0.006]),
        color=alt.Color("method")
        .scale(
            domain=["π (DMP, s_eff≈s0)", f"π (DMP, s_eff≈{s_eff_estimate:.2f})"],
            range=["#AAA", "seagreen"],
        )
        .legend(
            title="Method",
        ),
    )
).properties(
    title=alt.TitleParams(
        "DMP Approximation π with estiamted s_eff",
        subtitle="The DMP approximation gets more accurate when using the estimated s_eff",
        subtitleColor="#666",
    )
)

alt.LayerChart(...)